In [1]:
import os
from selenium import webdriver
from bs4 import BeautifulSoup
from pandas import DataFrame
from selenium.webdriver.common.keys import Keys 
import time 
import pyperclip
import re
import datetime


#####로그인하기(우회해서)
driver = webdriver.Chrome('C:\\chromedriver_win32\chromedriver')


uid = 'ieese12' 
upw = 'nadnr!23' #네이버 로그인 페이지로 이동 
 

url = ('https://nid.naver.com/nidlogin.login')
 

driver.get(url) 
time.sleep(2) #로딩 대기 


#아이디 입력폼 
tag_id = driver.find_element_by_name('id') 
#패스워드 입력폼 
tag_pw = driver.find_element_by_name('pw') 
 

 

# id 입력 
# 입력폼 클릭 -> paperclip에 선언한 uid 내용 복사 -> 붙여넣기 
tag_id.click() 
pyperclip.copy(uid) 
tag_id.send_keys(Keys.CONTROL, 'v') 
time.sleep(1)

 

# pw 입력 
# 입력폼 클릭 -> paperclip에 선언한 upw 내용 복사 -> 붙여넣기 
tag_pw.click() 
pyperclip.copy(upw) 
tag_pw.send_keys(Keys.CONTROL, 'v') 
time.sleep(1) 


 

#로그인 버튼 클릭 
login_btn = driver.find_element_by_id('log.login') 
login_btn.click() 
time.sleep(2)
 

 

######네이버 카페로 이동
driver.get('https://cafe.naver.com/casuallydressed')
 

#페이지 이동 함수 구축
#검색어 인코딩 및 문자치환하기
keyword = input()
keyword_ogn = keyword
keyword = keyword.encode('MS949')
keyword = str(keyword)
keyword = keyword.replace('\\x', '%')
keyword = keyword[2:-1]
 

#페이지이동 함수
def move_page(keyword,page):
    key_url = 'https://cafe.naver.com/casuallydressed?iframe_url=/ArticleSearchList.nhn%3Fsearch.clubid=19943558%26search.media=0%26search.searchdate=all%26search.defaultValue=1%26search.exact=%26search.include=%26userDisplay=15%26search.exclude=%26search.option=0%26search.sortBy=date%26search.searchBy=1%26search.includeAll=%26search.query={}%26search.viewtype=title%26search.page={}'.format(keyword,page)
    return key_url


data = [] 
 
# 크롤링 / 리눅스기준 한 페이지 긁어오는데 약 1분 10초
for page in range(1,35):
    url = move_page(keyword, page)
    driver.get(url)
         # 프레임 변경
    driver.switch_to.frame('cafe_main')

    n = 1 
    
    search_url = driver.page_source
    soup = BeautifulSoup(search_url, 'html.parser')

          # 제목 추출
    subj_locate = '#main-area > div:nth-child(n) > table > tbody > tr:nth-child(n) > td.td_article > div.board-list > div > a.article'
    subjects = soup.select(subj_locate)

    for subject in subjects:
        d = []
        
        try:
            ##날짜
            date_locate = '#main-area > div:nth-child(7) > table > tbody > tr:nth-child({}) > td.td_date'.format(n)
            date = soup.select(date_locate)
            date = date[0].getText().strip()

            #오늘 작성한 게시글은 날짜가 아니라 오늘 작성한 시간이 나온다.
            #이를 오늘날짜로 바꿔준다.
            if len(date) <= 5:
                date = datetime.date.today().strftime( "%Y.%m.%d." )

            d.append(date)
            d.append( '|||' )


            ## 제목
            sub = subject.text.strip()
            d.append( sub )
            d.append( '|||' )
    
        
            ##내용
            content_link = subject.attrs['href']
            content_link = 'http://cafe.naver.com' + content_link
            driver.get(content_link)
                         #프레임 변경
            driver.switch_to.default_content()
            time.sleep(1)
            driver.switch_to.frame('cafe_main')

            time.sleep(1)
            content = driver.find_element_by_xpath('/html/body/div/div/div/div[2]/div[2]/div[1]/div/div[1]/div/div').text
            content = content.replace('● 모든 게시물은 사진 + 글 첨부 필수 & 질문 게시글 삭제시 활동정지','').replace('● 모든 게시물은 사진 + 글 첨부 필수 https://cafe.naver.com/casuallydressed/44062 매뉴얼 확인 부탁드립니다.','').replace('\n','')
            d.append( content )
            
        except Exception as e:
            print( 'error 발생' )
            continue
       

        driver.back()                    

        #data에 집어넣기
        print( date, "|||", sub, "|||", content )
        data.append( d )

        n += 1

 

 
#이미 파일이 존재하면 존재하는 파일삭제
c = os.path.exists( 'wordcnt2.txt' )    
if c:
    os.remove( 'wordcnt2.txt' )

#텍스트파일로 저장
with open( 'wordcnt2.txt', 'w', encoding='utf-8' ) as f:
    for line in data:
        for l in line:
            f.write( l )
        f.write( '\n' )

아크테릭스
2021.07.18. ||| 아크테릭스 스쿼미시 많이 얇나요? ||| 파타고니아 토렌쉘 있는데 토렌쉘은 완전 얇은 바람막이보단 약간 두꺼운 재질인데 스쿼미시랑 안겹칠까요?
2021.07.17. ||| 요즘 일상? (993, 블레스, 클락스, 아크테릭스) ||| 이 날 펩시 제로를 처음 먹어봤는데 정말 맛있더라구요저처럼 여름에 꼭 긴바지에 후드티 입고 싶은 분 계신가요?더위도 완전 많이 타고 땀도 많은 체질인데 저 날 무슨 생각인지 모르겠지만 긴바지에 후드티를 입었네요 ㅋㅋ...생일 선물로 갖고 싶던 블레스 팔찌를 받았는데진짜 너무 예뻐서 맨날 차고 다닙니다••자주 매고 다니는 맨티스1 크기는 별로 안 크지만 들어갈건 다 들어가서 좋아요👍유니네 평소에 사고 싶던 바지 살까 말까 고민하다가 결국 샀는데 대만족하고 자주 입고 다니네요 흐흐집에만 있는 게 너무 힘들어서 카페가서 책 읽으려고 책 들고 동네 카페로 갔는데 등장인물 이름들이 너무 어려워서 얼마 못 읽고 덮어버렸습니다••• 날씨가 너무 좋길래 친구들 만나서 산책을 했는데 정말 더워 죽을 뻔 했습니다😵‍💫짧았을 때길었을 때고아캐 회원님들 짧은 머리가 너무 멋있으셔서 해봤는데 저랑은 안 어울리는 거 같아서 다시 기르는 중입니다🥲제가 가장 좋아하는 선수인데 이적설 돌아서 슬픈 요즘입니다쓰다보니까 맥락도 없는 글이 되어 버렸네요😵‍💫회원님들 모두 더위 조심하세요🙇‍♂️
2021.07.17. ||| 6-7월(베일런스,아크테릭스,로아) ||| 언제 봐도 이쁜 키코아식스베일런스 자켓 가볍고 좋아요살로몬 빔즈 많이 신네요비나 시원하게 내려줬으면 좋은데..여름 물회회덮밥시골 사는 친구 올라와서 동네 술집에서 술한잔 했네요하이킹? 할때 신을려고 산 로아!내소사언제나 아크테릭스변산반도!로아 착화감 좋았어요자켓 아크테릭스하의 acg신발 로아가방 아크테릭스할머니 기일이여서 온 산소 내돈내산머렐도 즐겨 신어요@ssik_99
2021.07.17. ||| 혹시 다들 아크테릭스 착샷 있으실까요 ?! ||| 제타 베타 매장가서 

2021.06.15. ||| 아크테릭스 제타sl 감마sl 둘중 뭐 사야할까요? ||| 8월달 쯤에 살 예정인데 입문으로 제타sl 감마sl중 뭘골라야할지 선택하기 힘드네요 제타는 방수 되고 감마는 안되는 걸로 아는데 착용감은 둘중 뭐가 더 괜찮고 8월 달 중순에 사면 뭘 사는게 좋을지 모르겠습니다 괜찮은 제품 추천 부탁합니다..(사용 후기도 괜찮으시다면 부탁 드려요 🙏🙏)
2021.06.15. ||| 아크테릭스같은 룩을 뭐라고 하나요? ||| 아웃도어나 바람막이로 코디하는 것을 뭐라고 하나요?? 기억이 안납니다ㅜ
2021.06.15. ||| 아크테릭스 노딘 사이즈 + 각종 질문합니다! ||| 스펙: 179/67아톰 lt로 입문하여 타운용+산 오를 때 잘 입고 다니고 있습니다. 지금 입기 좋은 걸로 하나 구입하려고 하는데 스쿼미시, 인센도, 감마sl 셋 중에 고민하다가 노딘이 입었을 때 감기는 느낌도 좋고 주머니도 있고 꽤나 추천을 많이 하시는 거 같더라고요. 그래서 노딘으로 가려고 하는데 아톰lt랑 같은 똑같은 M 사이즈 가도 무방할까요? 그리고 고아캐 분들은 노딘 어떤 색상 있으신지 궁금해유 갠적으로 21ss Kinetic 이게 젤 이뻐보이는 거 같음요
2021.06.15. ||| 20210614 아크테릭스 ||| 상의 :  아크테릭스 베일런스            하의 :  라츠신발 :  나이키악세 :          키/몸 :  184. 70내용 : 카페 몇군데 마실다녀왔습니다.ㅎㅎ엄청덥지만 자켓입어도 큰차이 못느낄정도 입니다비몇방울와서 맞았는데 고어텍스의 힘.. 흘러내리네요 ㅎㅎ기존 크림진 오염되어서 새로 풀카운트를 영입하려했지만 통장사정으로 라츠삿는데 겐춘해요
2021.06.14. ||| 안녕하세요 아크테릭스 정가품 여쭙습니다 ||| 아크테릭스 역시즌으로 제타 입문했는데 정품으로 보여지긴하는데 혹시 몰라 여쭙습니다 ㅎㅎ..하루 잘 마무리하시길바랍니다:)
2021.06.14. ||| 아크테릭스 바지 아시는분 있으신가요? ||| 어제 카페에서 이 제품 입으신

2021.05.26. ||| 아크테릭스 잘 아시는분 도움좀 부탁드립니다. (아톰 LT 자켓) 사이즈 문의 ||| 안녕하세요 고아캐 첫 글이 질문글 이네요 ㅠㅠ정말 어릴때 아크테릭스 입어보고 파타고니아도 어릴때 20대에 입어보고 정작 지금 30후반에는 갑자기 아크테릭스 꽂혀서 아톰 LT 자켓 구매하려는데이게 후드 넌후드 제품이 있는데 활용도 면이나 사이즈좀 추천부탁드릴게요시즌이 아니라서 백화점에서 입어볼수도 없고 ㅠㅠ 말들이 다 달라서 혼돈의 카오스 입니다 ㅠㅠ187 84이며 지금은 제가 몸이 좀 아파서 78키로까지 내려왔는데 보통 국내 105 주로 입고 아우터는 110을 입습니다유니클로 XL 띠어리 XL 국내 아웃도어 패딩들 보통 105-110유로 52 사이즈 스톤 모든 상의류 아우터 XL 이런데 ... 아톰 LT 라지 갈지 엑스라지 갈지 종결해주세요 ㅠㅠ지인의 M 을 입었는데 진짜 광배를 감싸는 뭔가 싸이클 선수들 쫄쫄이 옷 같아서 ㅠㅠ 사이즈 편차가 크면 라지가는게 맞는데 애매해서 지금 라지갈지 엑스라지 갈지 고민입니다.보통 그리고 저렴할때 직구가가 얼마정도 할가요? 몇달러? 정도인가요?
2021.05.25. ||| 트래비스 스캇의 아크테릭스 착장 ||| 이쁘네요 
2021.05.25. ||| 아크테릭스 백팩 고민... ||| 등산 및 일상 생활에서의 백팩으로 아크테릭스의세브링과 아로22 중 고민중입니다.둘다 고만고만해서 고민이 크네요 ㅠㅠ세브링으로 마음이 기울었다가다시보니, 세브링은 양쪽 물통 넣는 포켓이 없는 것 같은데없는게 맞는 건가요?혹시 아시는 분이 있으실지...무기명 투표 중등산 및 일상용 백팩세브링아로22투표하기
2021.05.24. ||| 아크테릭스 감마sl 사이즈 질문드려요 ||| 아크테릭스 감마sl를 구매하려고 합니다 키170/몸무게65인데 s사이즈가 맞겠죠??아크테릭스 첫구매라 질문드립니다!! 
2021.05.22. ||| 아크테릭스 아톰lt 색상추천! ||| 아크테릭스 아톰lt 많은분들이 추천해주셨어서 미리 구매하려고 하는데요!!색상이

2021.05.02. ||| 아크테릭스 베타 ||| 요즘 미친듯한 날씨에 아크테릭스 입으신분들이 많이 보이던데 너무 이뻐 인터넷 검색중 웍스아웃에 라스트원 제품 일단 주문 넣어 놨는데 지금 구매 하기엔 너무 늦었겠죠 ? 적은 금액이 아니라 고민 되네요 ㅠㅠ
2021.05.01. ||| 아크테릭스 제타sl 팀브레, 머렐 하이드로목 구입 후기 ||| 안녕하세요. 오랜만에 왔습니다. 최근 구매한 것들 중에 상당히 만족하는 두 제품을 입고 동네 마실 다녀왔습니다😁평소 제타 팀브레 컬러를 구매하고 싶어서 항상 생각하던 도중 사이즈가 한장 남은 곳이 있어서 바로 구매했네요ㅎㅎ 185/90 제타sl xl사이즈 딱좋네요.허나, 어제 오늘같이 선선한 날은 좋겠으나 여기서 더 따뜻해지면 더울거 같아요ㅠ 머렐도 상당히 만족하며 신고 있습니다.제타는 다음에 산행에서도 입어보고 싶네요.곧 재밌는 일상 얘기로 다시 오겠습니다! 그럼 이만🙇🏻🙇🏻
2021.04.30. ||| 포터 대용으로 아크테릭스 ||| 포터보다 튼튼할것같기도 하고 방수되는게 마음에 드네요다음달 지출이 벌써 정해졌습니다
2021.04.29. ||| 아크테릭스 스토우 팬츠 색상 ||| 타운 겸 등산용입니다 ㅎ무슨 색상이 좋을까요?끌리는 대로 추천 부탁드려요.^^1.2.
2021.04.28. ||| 아크테릭스 바지 질문드립니다!! ||| 안녕하세요!!사진 속 바지 모델이 혹시 뭔지 알고 계신 분 계신가요?? 아크테릭스를 아직 잘 알지못해서 막눈인 제가 봤을 땐 바지 모델들이 다 비슷비슷 한거 같아서 고아캐 회원분들께 여쭤봅니다
2021.04.28. ||| 아크테릭스 컨테이어 벨트 일상용으로 어떤가요? ||| 벨트,,, 귀찮고 무겁고 걸리적거려서 싫어하는데요허리 안 맞는 바지가 많아서 벨트는 차긴 차야겠더라구요 ㅠ 수선 알아보니 개당 1.5~2,3마넌이라 엄두가 안 나고요..이 벨트 일상용으로 어떻게 보세요?넣입 안 하면 괜찮을 거 같은데 흐음 ㅠ
2021.04.28. ||| 아크테릭스 어디서 직구하시나요?? ||| 안녕하

2021.04.10. ||| 아버지 아크테릭스 선물 모델 ||| 안녕하세요알바해서 모은 돈으로아버지께 아크테릭스 자켓 하나 선물해드리려는데어떤 모델이 좋을까요?평소에 등산을 무척 좋아하시긴 하는데찾아보니 등산용 일상용이 나뉘는 것 같더라고요두께별로 sl lt ar 등등도 있구요등산과 일상에서 둘다 활용하기 좋은 모델은 어떤모델이라고 생각하시나요??꼭 지금의 날씨에서 생각하지 않고일년 내내로 활용도 좋은게 무엇일지..옷이 많지 않으셔서 등산 때도 좋고일상에서도 자주 입을 수 있으면 좋겠습니다 ㅠㅠ
2021.04.10. ||| 아크테릭스 무슨 제품인가요? ||| 아크테릭스 입문 하려는데 무슨 제품인지 알려주실 분 계신가요!!
2021.04.10. ||| 아크테릭스 레프로이 vs 스토우 둘중 뭐가 좋을까요 ||| 등산보다는 일상생활 용으로 입으려고 합니다고프코어나 캐주얼한 복장으로 입으려고 하는데요두 바지 중 뭐가 더 괜찮은지 궁금합니다둘 다 조금 핏한 느낌으로 알고 있는데 평소 사이즈대로 입으면 너무 핏할까요? 섣불리 업했다 아재바지 느낌 날까 염려되네요
2021.04.09. ||| 아크테릭스 팬츠 추천 해주세요 ! ||| 등산 목적 & 데일리 느낌으로 입고 싶어요 !그리고 와이드 핏이였으면 좋겠습니다,,
2021.04.09. ||| 아크테릭스 구매했습니다. (입문자) ||| 제품명은 프레이저 자켓 입니다.일상에서 입을 수 있게 제작된 라인 이라고 설명 하셨고, 베타 랑 고어텍스는 같은거 썼다고 하더라구요.일단 앞기장 뒷기장이 같은게 마음에 들었습니다가슴에 로고는 없고 팔쪽에 프린팅 되어있어서 좀 더 데일리로 입기 좋아 보였습니다!가격은 10프로 할인 받아 621,000원 이였네용평소 100,L 입는데 라지로 구매했습니닷
2021.04.09. ||| 아크테릭스 LEAF ||| 믿음직스러운 전천후 자켓
2021.04.09. ||| 아크테릭스 제품 이름 알수있을까요? ||| 아크테릭스 제품 이름 알수있을까요 ?
2021.04.08. ||| 아크테릭스 베타 LT 구했네

...

2021.02.14. ||| 빔즈 아크테릭스 제타 올해모델과 작년모델중 어느게더 나을까요? ||| 둘다 너무 이쁜데 둘다 들이는건 오바같아서 하나만 선택하려하는데 둘다 다른매력이 있네요
2021.02.14. ||| 아크테릭스 베타 lt 궁금증 ㅠㅠ ||| 첫 아크테릭스를 베타 lt로 구매하였습니다. 제타와 많이 고민했지만 등산용보다는 패션용이 커서 짧은 기장에 눈이 갔습니다.제가 이런 기능성옷은 처음이라 그런데 고어텍스 자켓이 자켓 특성상 오염에 취약하다 들었습니다. 그레이색 특성상 오염티가 많이 날텐데 검정으로 바꿔야하나 고민중입니다 ㅠㅠ 그리고 lt는 되게 얇은 자켓으로 들었는데 어느정도인지 감이 안잡힙니다. 나일론 코치자켓정도 얇기로 생각하면 될까요?
2021.02.12. ||| SS21 아크테릭스 베타LT 사이즈 질문 드립니다. ||| 20시즌 아톰LT 후드없는제품 L사이즈 딱맞게 입는 177-78 헬스인 입니다베타LT 21시즌 제품이 공홈에서 구입하려하는데 M/L/XL 뭐로 가야할지 고민입니다.혹시 고수분들있으시면 사이즈 추천 부탁드립니다
2021.02.12. ||| 아크테릭스 ? ||| 이 모델이 입은 아크테릭스 알파제품인가요?,, 바지 무슨 제품인지 궁금합니다!
2021.02.12. ||| [질문] 아크테릭스 타운용으로 더 적합한것은? ||| (위) 솔라노 VS (아래) 아톰LT 찾으면 찾을수록 고민이 큽니다...
2021.02.11. ||| [어미새] 아크테릭스 아톰 lt 후디 ||| 캠프세이버 대란급은 아니지만 가격 좋네요rei구요 블랙은 할인안하지만 파라독스 포함 세 가지 색상 할인중입니다. 
2021.02.11. ||| 아크테릭스 솔라노 질문... ||| 이게 무슨 색상일까요... 어렵네요
2021.02.10. ||| 백컨트리 아크테릭스 세일하네요 ||| vpn쓰고 접속해야 결제되는 좀 어려운 사이트입니다. (저도 아직 시도 못해본곳입니다ㅜ)https://www.backcountry.com파이어비 올리브색 가격 좋은거 같네요. 구매하실때 리

2021.01.30. ||| 아크테릭스 베타lt 자켓 올해제품 나왔네요. ||| 롯데백화점 아크테릭스 구경왔는데, 베타sl하이브리드, 베타lt, 제타sl 올해제품 출시됐네요. 사이즈별로 10개정도 재고 있다네요.세제품 모두 미디움 입어보고, 베타lt 블랙으로 구매했습니다. 베타lt 제품이 베타sl, 제타sl보다 넉넉하게 나왔습니다.
2021.01.30. ||| 첫 아크테릭스 제품 후기 입니당 ||| 아톰LT을 구입하고 싶었으나 사이즈나 색상이 없는 관계로 대체품으로 프로톤LT로 구입하였습니다175/66 기준으로 넉넉하게 입을려고 L사이즈로 구입할려고 했으나 등산용으로 바람이 들어오지 못하게 빈틈을 없애기 위해 M사이즈 정핏으로 구매했습니다.색상은 약간 톤 다운된 주황색이라고 생각하시면 될 것 같아요.저만의 피셜인데 타 아웃도어 비해 확실히 옷의 질감이나 브랜드 로고가 주는 감성은 확실히 다른것 같습니다 이상입니다(막상 등산하면 옷이 거기서 거기지 더러워지는건 똑같은데 ㅠㅠ)
2021.01.30. ||| 빔즈 아크테릭스 제타 SL 사이즈 질문! ||| 빔즈 아크테릭스 제타 sl 구매대행해서 구입하려는데 제타 sl과 같은 사이즈 사면 될까요??아니면 사이즈 추천좀 부탁드립니다!174/73이고 평소 102-103 입습니다.M과 L 중 어떤걸 사야 적당히 여유있게 잘맞을까요?
2021.01.29. ||| 아크테릭스 X 빔즈 별주 제타 SL 프리오더 ||| https://www.beams.co.jp/item/beams/blouson/11186112729/BEAMS（ビームス）【予約】ARC’TERYX × BEAMS / 別注 Zeta SL Jacket（ブルゾン ブルゾン）オールグレーでまとめた別注ワントーンコレクションwww.beams.co.jp큰 사이즈들 은 나갓어요
2021.01.29. ||| H&M 아크테릭스느낌(죄송합니다)의 발수코팅자켓 2.7발 ||| 할인이  오늘 오전9시까지로 알고있고블랙컬러기준H&M회원가입을 생일 1.29로 설정후 가입해서생일쿠폰확보하고할인코드 831

2021.01.19. ||| 아크테릭스 제타 ||| 첫 아크테릭스 입문 :)지금 입기는 춥지만 조금만 날이 따뜻하면 잘 입을 것 같습니다 
2021.01.18. ||| [후기] 아크테릭스 X 팔라스 멘티스, 볼캡 후기 및 간단 착샷 |||  재고 빠지기 전에 후다닥 구매한 아크테릭스 팔라스 후기입니다.앞서 구매하신 분들이 많이들 후기 남겨주셔서고아캐와 가장 어울리는 아이템이지만, 후기는 제일 늦게 작성하네요.간단하게 남겨 보겠습니다.사실 아크테릭스라는 브랜드가 생소하긴 하지만 지금 가장 핫한 브랜드중 하나가 아닐까 싶습니다.그래서 팔라스 아크테릭스 협업 또한 굉장히 화제가 되었으나,서로의 아이템에 로고만 빌려준 듯한 협업은 어딘가 허전하면서 심심한 느낌입니다.오히려 이런 협업이 서로의 아이덴티티를 잘 살렸다 라고 보시는 분들도 있었습니다. 상세사진노란색 포인트가 괸장히 강렬하네요.모자는 퀄리티로 따지면 딱 58불짜리 느낌이었습니다.제가 팔라스 모자를 여러개 사서 써봤는데이 제품 또한 팔라스 핏이였습니다.얕습니다.아크테릭스 마카 제품을 써봤는데 활용도가 떨어져서 매물하였었습니다.빔즈 콜라보 제품을 알아보다가 팔라스 협업 제품이 나와서 구매해보았습니다.맨티스는 생각보다 작군요.. 어떤식으로 활용해야할지..사실 가격으로 따지는 것은 의미 없는 것 같고두 브랜드가 만난 것 만으로 유의미하지않나 싶습니다.노란색 포인트가 멋지고,그 자수가 아크테릭스와 팔라스 로고이고너무 좋네요.착샷아우터 : 라이풀상의 : 폴로하의 : 유니네신발 : 척테일러가방 : 아크테릭스 팔라스모자도 착용 하였으나 안면부가 굉장히 복잡한 관계로 비공개입니다. 양해 바랍니다.존밤되세요!
2021.01.18. ||| 약미새) 아크테릭스 ||| 녹색창 검색하다가 봤네요.필요하신 분들 소장하세요!물량 많이 빠졌네요;
2021.01.18. ||| 아크테릭스 베타 ar 질문 ||| 아크테릭스 한번 구매해보고 싶어서 구매해볼려고 하는데 베타 ar은 사이즈를 어떻게 가야하나요? 181에 83정도입니다 .그리고 혹시

2021.01.06. ||| 2020년 마지막 쇼핑! (아크테릭스 제타 SL 팀브레) ||| 2020년 12월 중순쯤 세일하길래 구매해서12월 31일 마지막날 받았네요.고아캐드에서 아크테릭스 관련 글 많이 봤었는데 등산도 하지않는 저에겐 가격이 너무 비싸 진입하기 힘들었는데50프로 세일에 혹해서 아크테릭스 처음 발들이네요~가격은 50프로 할인해서 149.5달러에 구매했고 배대지 비용 12,000원해서 총 175,000원 정도에 구매했어요!인터넷으로 알아봤을때보다 싸게산거같아요~근데 이렇게 싼 이유가 사이즈가 XXL라서 그런거같아요카페글을 찾아보니 제타는 기장이 길고 특히 뒷기장보통 사이즈보다 한사이즈 다운해서 가시더라고요.제가 187/85인데 이 스펙에서 보통 L사이즈를 가시던데 XXL를 구매해서 너무 크지않을까 우려가 많았는데역시나 엄청 크네요팔길이랑 품이 엄청 큰데 조절할 수 있는 부분이 있어 불편하지는 않게 입을 수 있을거같아요~저는 등산은 하지 않아 등산복 용도말고 봄가을 그냥 외투로 입고 가볍게 입으면 딱 좋겠네요~~
2021.01.06. ||| 아크테릭스 베타 ar 정가품 문의드려요 |||  https://cafe.naver.com/casuallydressed/44062 매뉴얼 확인 부탁드립니다. 구매하게되었는데 정가품좀 봐주셨으면 정말 좋겠습니다ㅜㅜㅜㅜ
2021.01.06. ||| 아크테릭스 제타 sl 정가품 질문... ||| 새상품으로 해외직구 했는데 혹여나 가품일지 걱정되어 질문합니다...  https://cafe.naver.com/casuallydressed/44062 매뉴얼 확인 부탁드립니다. 
2021.01.05. ||| 어제 받은 팔라스 x 아크테릭스 세륨 LT 후디입니다. ||| 어제 영팔에서 받은 팔크테릭스 세륨 LT입니다~콜라보 한정판 제품을 좋아합니다.가볍긴 정말 가볍네요!172 70인데 m사이즈 적당하게 잘 맞네요! 등산&타운용으로 입으려고요오늘도 좋은 하루 되세요~~
2021.01.04. ||| 아크테릭스 파이어비 24K 골드

2020.12.22. ||| 아크테릭스 파이어비 24k 블랙 모델은 전부 자수로고 + 고어텍스 인피니움 인가요? ||| 국내는 다 품절이라 직구 하려고 하는데 24k 블랙 모델은 전부 자수로고에 인피니움 사용하는것으로 아는데이렇게 사진 처럼 24k 블랙인데 써미움에 자수가 아닌거 같아서요...올블랙 모델의 경우는 구형이라 확실히 자수가 아니고 써미움 사용한것으로 아는데 여지껏 24k 블랙은 자수모델인걸로 알았거든요24k 블랙 모델 자수가 아닌건 2018년도 이전에 만든건가요?답변 감사합니다
2020.12.21. ||| 오늘 아침8시에 도착한 팔라스 아크테릭스~ ||| 방금전 도착해서 기쁨 맘에 반팔티랑 매치해봤어요구성은 알차게 되어있어요에어팟이랑 아이폰 그리고 자동차키 신용카드등등간편하게 수납할수있어 실용적입니다^^
2020.12.20. ||| 아크테릭스 아톰lt ||| 타운용으로 입기 어떤가요?? 등산용이라는 느낌이 강한지 모르겠네요.지금처럼 혹한기(?)에는 외투 안에 레이어링하기에 적절한가요?
2020.12.20. ||| 아크테릭스 아톰lt ||| 회원님들 아크테릭스 아톰lt 대략 30만원정도의 가치 충분히 한다고 생각하시나요??보통 옷을 살 때 직구등을통해 최대한 싸게 살려고 노력하는편인데아크테릭스 아톰lt혹은 ar 너무 사고싶어 찾아보는데애매하게 207불 까지밖에 못찾겠네요..관부가세 때문에 약 6만원정도 더 내겠네요이 돈이라도 주고 사야할지혹은 대체제 (ex 스쿼미시 등 저렴라인)을 사 입다가할인할때 관세이하로 노려야 할지 고민이 됩니다 ㅠ
2020.12.19. ||| 아크테릭스 기본~~ ||| 적당한 날씨에는 아크테릭스좋아요~~다들 감기 조심 하세요아크테릭스 리프 면티아크테릭스 리프 아톰아크테릭스 토륨24K버버리 캡스톤 팬츠프라다 슈즈
2020.12.17. ||| 아크테릭스 겨울 레이어링 질문드려요 ||| 안녕하세요아크테릭스를 제타 sl로 입문했습니다얼마전까지는 편하게 입었는데 이제 날씨가 추워져서 안에 니트를 입어도 한기가 올라오더라구요그래서 아톰